In [ ]:
import copy

import prody
from rdkit import Chem
from rdkit.Chem import Draw

from rgroup.replace import rep2D, draw3D, merge_R_group

In [ ]:
# load the complex with the ligand
sys = prody.parsePDB('5NEV.pdb1')
rec = sys.select('not (nucleic or hetatm) and not water')
mol = sys.select('resname 72L')

prody.writePDB('rec.pdb', rec)
prody.writePDB('lig.pdb', mol)

In [ ]:
# load the ligand
mol = Chem.MolFromPDBFile('lig.pdb')
mol = Chem.AddHs(mol, addCoords=True)
mol

In [ ]:
# select the atom to be replaced with the R group
Draw.MolToImage(rep2D(mol))

In [ ]:
# load the R group
R_group = Chem.MolFromMolFile('data/rgroups/molecules/alcohols/methanol-r.mol', removeHs=False)
viewer = draw3D(mol)

In [ ]:
finalMol = merge_R_group(mol, R_group, 72)
finalMol

In [ ]:
Draw.MolToImage(rep2D(finalMol))